Mock CRISI BCA – Applicant Submission Workbook Construction

This notebook constructs a simplified benefit-cost analysis workbook representing an illustrative CRISI grant applicant submission.

All data are synthetic and created solely for methodological demonstration.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Project Parameters
project_life = 20
discount_rate = 0.07

capital_cost = 82_000_000
annual_om_cost = 1_500_000

In [3]:
# ---------------------------------------
# Create the year index (0..20)
# and compute discount factors for PV.
# Applicant assumes 7% real discount rate.
# ---------------------------------------

years = np.arange(0, project_life + 1)  # years 0 through 20

df = pd.DataFrame({"year": years})

# Present Value (PV) discount factor: 1 / (1+r)^t
df["discount_factor"] = 1 / ((1 + discount_rate) ** df["year"])

In [4]:
# ---------------------------------------
# Construct applicant cost streams.
# - Capital cost incurred upfront in year 0.
# - O&M incurred annually in years 1..20.
# ---------------------------------------

df["capital_cost"] = 0.0
df.loc[df["year"] == 0, "capital_cost"] = capital_cost

df["om_cost"] = 0.0
df.loc[df["year"] >= 1, "om_cost"] = annual_om_cost

df["total_cost"] = df["capital_cost"] + df["om_cost"]

In [5]:
# ---------------------------------------
# Discount annual costs to PV and
# compute total PV of costs (PVC).
# ---------------------------------------

df["pv_cost"] = df["total_cost"] * df["discount_factor"]
pv_cost_total = df["pv_cost"].sum()

pv_cost_total

np.float64(97891021.36827423)

In [6]:
# ---------------------------------------
# Check the first few rows.
# Expect:
# - Year 0: capital_cost = 82M
# - Years 1+: om_cost = 1.5M
# - discount_factor declining over time
# ---------------------------------------

df.head(6)

,year,discount_factor,capital_cost,om_cost,total_cost,pv_cost
0,0,1.000000,82000000.0,0.0,82000000.0,8.200000e+07
1,1,0.934579,0.0,1500000.0,1500000.0,1.401869e+06
2,2,0.873439,0.0,1500000.0,1500000.0,1.310158e+06
3,3,0.816298,0.0,1500000.0,1500000.0,1.224447e+06
4,4,0.762895,0.0,1500000.0,1500000.0,1.144343e+06
5,5,0.712986,0.0,1500000.0,1500000.0,1.069479e+06


In [7]:
# ---------------------------------------
# Applicant Assumptions (As Submitted)
# ---------------------------------------

# Safety assumptions
baseline_incidents = 6
incident_reduction_rate = 0.35
value_statistical_life = 9_600_000  # Applicant uses outdated VSL

# Travel time assumptions
annual_travel_time_benefit_year1 = 14_200_000
traffic_growth_rate = 0.04  # 4% annual growth (aggressive assumption)

# Vehicle operating cost savings
annual_voc_benefit_year1 = 3_500_000

# Emissions benefit
annual_emissions_benefit_year1 = 1_900_000

In [8]:
# ---------------------------------------
# Construct Annual Benefit Streams
# ---------------------------------------

df["safety_benefit"] = 18_400_000  # flat annual safety benefit (applicant simplification)

# Travel time grows at assumed traffic growth rate
df["travel_time_benefit"] = 0.0
for t in df["year"]:
    if t >= 1:
        df.loc[df["year"] == t, "travel_time_benefit"] = \
            annual_travel_time_benefit_year1 * ((1 + traffic_growth_rate) ** (t - 1))

# VOC grows with traffic
df["voc_benefit"] = 0.0
for t in df["year"]:
    if t >= 1:
        df.loc[df["year"] == t, "voc_benefit"] = \
            annual_voc_benefit_year1 * ((1 + traffic_growth_rate) ** (t - 1))

# Emissions also grow
df["emissions_benefit"] = 0.0
for t in df["year"]:
    if t >= 1:
        df.loc[df["year"] == t, "emissions_benefit"] = \
            annual_emissions_benefit_year1 * ((1 + traffic_growth_rate) ** (t - 1))

In [9]:
# ---------------------------------------
# Aggregate and Discount Benefits
# ---------------------------------------

df["total_benefit"] = (
    df["safety_benefit"]
    + df["travel_time_benefit"]
    + df["voc_benefit"]
    + df["emissions_benefit"]
)

df["pv_benefit"] = df["total_benefit"] * df["discount_factor"]
pv_benefit_total = df["pv_benefit"].sum()

pv_benefit_total

np.float64(496727661.77630764)

In [10]:
# ---------------------------------------
# Compute Applicant-Reported BCR
# ---------------------------------------

bcr = pv_benefit_total / pv_cost_total
bcr

np.float64(5.074292359332697)

In [11]:
# ---------------------------------------
# Calibrate benefits so the applicant's
# reported BCR matches the target (2.15).
# We scale ALL benefits by a single factor to
# preserve relative composition.
# ---------------------------------------

target_bcr = 2.15

scale_factor = target_bcr / bcr  # bcr is your current computed BCR
scale_factor


np.float64(0.42370440009151134)

In [12]:
# ---------------------------------------
# Apply scaling to each benefit stream
# and recompute PV benefits + BCR.
# ---------------------------------------

benefit_cols = ["safety_benefit", "travel_time_benefit", "voc_benefit", "emissions_benefit"]

for col in benefit_cols:
    df[col] = df[col] * scale_factor

df["total_benefit"] = df[benefit_cols].sum(axis=1)
df["pv_benefit"] = df["total_benefit"] * df["discount_factor"]

pv_benefit_total = df["pv_benefit"].sum()
bcr = pv_benefit_total / pv_cost_total

pv_benefit_total, pv_cost_total, bcr

(np.float64(210465695.94178957),
 np.float64(97891021.36827423),
 np.float64(2.15))

In [14]:
# ---------------------------------------
# Clean reporting summary
# ---------------------------------------

print("Present Value of Benefits: ${:,.0f}".format(pv_benefit_total))
print("Present Value of Costs:    ${:,.0f}".format(pv_cost_total))
print("Benefit-Cost Ratio (7%):   {:.2f}".format(bcr))

Present Value of Benefits: $210,465,696
Present Value of Costs:    $97,891,021
Benefit-Cost Ratio (7%):   2.15


Applicant-Reported Results (Calibrated)

Present Value of Benefits: $210,465,696

Present Value of Costs:    $97,891,021

Benefit-Cost Ratio (7%):   2.15



In [16]:
# ---------------------------------------
# Export a mock applicant submission workbook
# - Inputs (project + BCA assumptions)
# - CashFlows (year-by-year costs/benefits + PV)
# - Summary (PVC, PVB, BCR)
# ---------------------------------------

from pathlib import Path
from openpyxl import load_workbook

out_path = Path("applicant_submission.xlsx")

# Build Inputs sheet
inputs = pd.DataFrame(
    {
        "Item": [
            "Project life (years)",
            "Discount rate (real)",
            "Capital cost (Year 0)",
            "Annual O&M (Years 1..T)",
            "Traffic growth rate",
            "Applicant VSL",
        ],
        "Value": [
            project_life,
            discount_rate,
            capital_cost,
            annual_om_cost,
            traffic_growth_rate,
            value_statistical_life,
        ],
        "Units / Notes": [
            "years",
            "decimal (real)",
            "USD",
            "USD per year (real)",
            "decimal",
            "USD (as used by applicant)",
        ],
    }
)

# Build Summary sheet
summary = pd.DataFrame(
    {
        "Metric": ["PV Benefits", "PV Costs", "BCR (7% real)"],
        "Value": [pv_benefit_total, pv_cost_total, bcr],
    }
)

# Build CashFlows sheet (what we actually export)
cashflows_cols = [
    "year",
    "discount_factor",
    "capital_cost",
    "om_cost",
    "total_cost",
    "safety_benefit",
    "travel_time_benefit",
    "voc_benefit",
    "emissions_benefit",
    "total_benefit",
    "pv_cost",
    "pv_benefit",
]
cashflows = df[cashflows_cols].copy()

# Write sheets
with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    inputs.to_excel(writer, sheet_name="Inputs", index=False)
    summary.to_excel(writer, sheet_name="Summary", index=False)
    cashflows.to_excel(writer, sheet_name="CashFlows", index=False)

# Re-open and format with openpyxl
wb = load_workbook(out_path)

# ---- Summary formatting ----
ws_summary = wb["Summary"]
ws_summary["B2"].number_format = '$#,##0'   # PV Benefits
ws_summary["B3"].number_format = '$#,##0'   # PV Costs
ws_summary["B4"].number_format = '0.00'     # BCR

# ---- Inputs formatting (values are in column B; data starts at row 2) ----
ws_inputs = wb["Inputs"]
ws_inputs["B2"].number_format = '0'        # project life
ws_inputs["B3"].number_format = '0%'       # discount rate
ws_inputs["B4"].number_format = '$#,##0'   # capex
ws_inputs["B5"].number_format = '$#,##0'   # O&M
ws_inputs["B6"].number_format = '0%'       # traffic growth
ws_inputs["B7"].number_format = '$#,##0'   # VSL

# ---- CashFlows formatting ----
ws_cf = wb["CashFlows"]

# discount_factor column is column B
for r in range(2, ws_cf.max_row + 1):
    ws_cf[f"B{r}"].number_format = '0.0000'

currency_cols = [
    "capital_cost",
    "om_cost",
    "total_cost",
    "safety_benefit",
    "travel_time_benefit",
    "voc_benefit",
    "emissions_benefit",
    "total_benefit",
    "pv_cost",
    "pv_benefit",
]

# Map using the exported 'cashflows' dataframe (not the earlier 'inputs/summary' df)
for col_name in currency_cols:
    col_idx = cashflows.columns.get_loc(col_name) + 1  # 1-based
    col_letter = ws_cf.cell(row=1, column=col_idx).column_letter
    for r in range(2, ws_cf.max_row + 1):
        ws_cf[f"{col_letter}{r}"].number_format = '$#,##0'

wb.save(out_path)

print(f"Saved formatted workbook to: {out_path.resolve()}")

Saved formatted workbook to: /home/kip/PyCharmMiscProject/applicant_submission.xlsx
